In [1]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [2]:
def mat_dbg(x):
    print(x.shape, ": \n", x)

In [3]:
class MLP(object):
    """
    NOTE: Matrix operations are modified from RBM file
    In particular, we have an an an input of n data points with
    dimension d as a d-by-n matrix
    """
    def __init__(self, data, num_hidden, learn_rate):
        # First row are the target values
        self.targets = data[0]
        self.data = data[1:,:]
        self.num_data = data.shape[1]
        self.num_visible = data.shape[0] - 1
        self.num_hidden = num_hidden
        self.learn_rate = learn_rate
        
        self.weights = np.random.rand(self.num_visible + 1, self.num_hidden)/500
        self.data = np.insert(self.data, 0, 1, axis = 0)
    
    def _sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
        
    def _activated(self,mat):
        activated = self._sigmoid(np.dot(self.weights.T,mat))
        return activated
    
    def _gradient(self,mat):
        w = copy.deepcopy(mat)
        for elt in np.nditer(w,op_flags=['readwrite']):
            elt[...] = elt - elt * elt
        return w
    
    def _feed_forward(self):
        data_copy = copy.deepcopy(self.data)
        for i in range(0,self.num_data):
            data_col = data_copy[:,i]
            data_col = np.reshape(data_col, (-1, 1))
            
            activated_output = (self._activated(data_col))[0]
            guess = 1 - 2*activated_output
            #print("Guess: ", guess)
            
            error = self.targets[i] - guess
            grad = (self._gradient(activated_output))[0]
            
            c = (error * self.learn_rate)[0]
            wt_change = c * grad * data_col
            
            self.weights = wt_change + self.weights
            self.weights = self.weights / (np.amax(np.absolute(self.weights)))
        
        #print("New weights: ",self.weights)            
        
    def _prop_backward(self):
        return None
    
    def _calc_error(self,mat):
        return None


In [70]:
# See www.cse.unsw.edu.au/~cs9417ml/MLP2/
dims = 4
data_pts = 300 * dims
sample_data = np.sign(1-2*np.random.rand(dims,data_pts))
sample_data[0] = (1 * sample_data[1] + 1 * sample_data[2]) 
# Need to write binary target values for data as well for MLP
mat_dbg(sample_data)
MLP_1 = MLP(sample_data,1,0.001)
print("Weights^T: \n",MLP_1.weights.T)
#print("Targets: ")
#print(MLP_1.targets)
print("Data sample: \n",MLP_1.data[1:5])

(4, 1200) : 
 [[ 2.  0. -2. ...,  0.  0.  0.]
 [ 1.  1. -1. ..., -1. -1. -1.]
 [ 1. -1. -1. ...,  1.  1.  1.]
 [ 1. -1.  1. ...,  1.  1. -1.]]
Weights^T: 
 [[ 0.00083429  0.0010595   0.00148689  0.00176546]]
Data sample: 
 [[ 1.  1. -1. ..., -1. -1. -1.]
 [ 1. -1. -1. ...,  1.  1.  1.]
 [ 1. -1.  1. ...,  1.  1. -1.]]


In [71]:
iterations = 10
for i in range(0,10):
    MLP_1._feed_forward()
#MLP_1._feed_forward()
print("Weights: \n", MLP_1.weights)
results = np.dot(MLP_1.weights.T,MLP_1.data)
print("Targets: \n", MLP_1.targets)
print("Results: \n", results)

Weights: 
 [[ 0.05163147]
 [ 0.91088098]
 [ 1.        ]
 [ 0.02916911]]
Targets: 
 [ 2.  0. -2. ...,  0.  0.  0.]
Results: 
 [[ 1.99168156 -0.06665666 -1.8300804  ...,  0.1699196   0.1699196
   0.11158138]]
